# **EletroMag - Medidor eletrônico de energia** #

In [ ]:
#Bibliotecas
from cmath import *
from numpy import linalg
import numpy as np

---
Na célula a seguir serão definidas os parâmetros do sistema já conhecidos.:


In [ ]:
R1=6
R2=3
R3=3
R4=6
L1=300e-3
L2=30e-3
L3=1
L4=1
M1=sqrt(L1*L2)
M2=sqrt(L3*L4)
f=60
V1=220

w=2*pi*f
XL1=1j*w*L1
XL2=1j*w*L2
XL3=1j*w*L3
XL4=1j*w*L4
XM1=1j*w*M1
XM2=1j*w*M2

---
Definindo as funções que serão utilizadas no projeto:

In [ ]:
def CalcularTransformadorDeCorrente(Uf, Rs,Rs):
    Z=np.array([[R1+XL1, -XM],[-XM, XL2+R2+Rc]])
    V=np.array([Uf,0])
    i=np.dot(linalg.inv(Z),V)
    return i[0], i[1]